In [1]:
import os 
import time
import copy
from tqdm import tqdm

import numpy as np 
import matplotlib.pyplot as plt
import scipy
import pandas as pd

from joblib import Parallel, delayed
from numba import jit, njit, prange

#################### Macros #############################
np.random.seed(0)

# Physical constants
K=12 # number of fermionic modes
J=4 # ~"energy scale"
Q=4 # order of coupling
N = 2*K # number of fermions
N_DIM = 2**K # Hilbert space dimensions

# Directory to save sample Hamiltonians
H_DIR = os.path.join("Simulated Hamiltonians", f"H4_K{K}_J{J}_Q{Q}")
os.makedirs(H_DIR, exist_ok=True)

N_SAMPLES = 160 # number of samples to generate
N_JOBS = 20 # number of jobs to run in parallel

################## Locals #############################
from hamiltonian_generation import make_H4_sparse

## 1. Define Hamiltonian

In [2]:
H4_func = make_H4_sparse(K, J, random_seed=0, precompute_pairs = True, precompute_quads = True)

# 2. Simulate N_SAMPLES

In [3]:
sigma_j = np.sqrt((J**2)*np.math.factorial(Q-1)/(N**(Q-1)))
js_all = [np.random.normal(0, sigma_j, size=tuple([N for i in range(Q)])) for j in range(N_SAMPLES+1)]

def process_i(i):
    H4_i = H4_func(js_all[i]).toarray()
    np.save(f"{H_DIR}/H4_{i+1}.npy", H4_i)

tic = time.time()
Parallel(n_jobs=N_JOBS)(delayed(process_i)(i) for i in tqdm(range(10, N_SAMPLES)))
duration = time.time() - tic
print(f"Duration: {duration//60} minutes")

100%|██████████| 150/150 [1:10:25<00:00, 28.17s/it]


Duration: 106.0 minutes
